<a href="https://colab.research.google.com/github/mkwak92/food_sales_predictions/blob/main/Food_Sales_Prediction_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1 Machine Learning (cont.)
- Matthew Kwak

- Before splitting your data, you can drop duplicates and fix inconsistencies in categorical data.* (*There is a way to do this after the split, but for this project, you may perform this step before the split)
- Identify the features (X) and target (y): Assign the "Item_Outlet_Sales" column as your target and the rest of the relevant variables as your features matrix.  
- Perform a train test split 
- Create a preprocessing object to prepare the dataset for Machine Learning
Make sure your imputation of missing values occurs  after the train test split using SimpleImputer.  

In [52]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(display='diagram')

filename= 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTjJvfknetCj7XU5ZzDHiFUvloXpJnaakjRJjE8U8r5JMRnIdE_HIeFr3ODrNsUi8djGxvuQThaJsl7/pub?gid=1975659408&single=true&output=csv'
df = pd.read_csv(filename, header=0)
df.head()
ml_df = df.copy()

- Before splitting your data, you can drop duplicates and fix inconsistencies in categorical data.* (*There is a way to do this after the split, but for this project, you may perform this step before the split)

In [53]:
ml_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [54]:
ml_df.duplicated().sum()

0

In [55]:
ml_df['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

In [63]:
ml_df['Item_Fat_Content'].replace({'Low Fat':0, 'low fat':0,'LF':0,'Regular':1,'reg':1},inplace=True)

In [64]:
ml_df['Outlet_Size'].unique()

array(['Medium', nan, 'High', 'Small'], dtype=object)

In [65]:
ml_df['Outlet_Size'].replace({'Small':0,'Medium':1,'High':2},inplace=True)

In [66]:
ml_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   int64  
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   float64
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(5), int64(2), object(5)
memory usage: 799.2+ KB


In [62]:
ml_df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [71]:
ml_df.describe()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000,6113.000000,8523.000000
mean,12.857645,0.352693,0.066132,140.992782,1997.831867,0.761819,2181.288914
std,4.643456,0.477836,0.051598,62.275067,8.371760,0.697463,1706.499616
min,4.555000,0.000000,0.000000,31.290000,1985.000000,0.000000,33.290000
25%,8.773750,0.000000,0.026989,93.826500,1987.000000,0.000000,834.247400
50%,12.600000,0.000000,0.053931,143.012800,1999.000000,1.000000,1794.331000
75%,16.850000,1.000000,0.094585,185.643700,2004.000000,1.000000,3101.296400
max,21.350000,1.000000,0.328391,266.888400,2009.000000,2.000000,13086.964800


- Identify the features (X) and target (y): Assign the "Item_Outlet_Sales" column as your target and the rest of the relevant variables as your features matrix.

In [72]:
X = ml_df.drop(['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales'], axis=1)
y = ml_df['Item_Outlet_Sales']

In [73]:
X

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,9.300,0,0.016047,Dairy,249.8092,1999,1.0,Tier 1,Supermarket Type1
1,5.920,1,0.019278,Soft Drinks,48.2692,2009,1.0,Tier 3,Supermarket Type2
2,17.500,0,0.016760,Meat,141.6180,1999,1.0,Tier 1,Supermarket Type1
3,19.200,1,0.000000,Fruits and Vegetables,182.0950,1998,NaN,Tier 3,Grocery Store
4,8.930,0,0.000000,Household,53.8614,1987,2.0,Tier 3,Supermarket Type1
...,...,...,...,...,...,...,...,...,...
8518,6.865,0,0.056783,Snack Foods,214.5218,1987,2.0,Tier 3,Supermarket Type1
8519,8.380,1,0.046982,Baking Goods,108.1570,2002,NaN,Tier 2,Supermarket Type1
8520,10.600,0,0.035186,Health and Hygiene,85.1224,2004,0.0,Tier 2,Supermarket Type1
8521,7.210,1,0.145221,Snack Foods,103.1332,2009,1.0,Tier 3,Supermarket Type2


- Perform a train test split

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

- Create a preprocessing object to prepare the dataset for Machine Learning Make sure your imputation of missing values occurs after the train test split using SimpleImputer.

In [77]:
cat_selector = make_column_selector(dtype_include='object')
num_cols = ['Item_Weight','Item_Visibility','Item_MRP','Outlet_Establishment_Year']
ord_cols = ['Item_Fat_Content','Outlet_Size'] #item fat content, outlet_size is ordinal from above 


In [76]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Weight                7060 non-null   float64
 1   Item_Fat_Content           8523 non-null   int64  
 2   Item_Visibility            8523 non-null   float64
 3   Item_Type                  8523 non-null   object 
 4   Item_MRP                   8523 non-null   float64
 5   Outlet_Establishment_Year  8523 non-null   int64  
 6   Outlet_Size                6113 non-null   float64
 7   Outlet_Location_Type       8523 non-null   object 
 8   Outlet_Type                8523 non-null   object 
dtypes: float64(4), int64(2), object(3)
memory usage: 599.4+ KB


In [87]:
most_frequent_imputer = SimpleImputer(strategy='most_frequent') #most frequent since I updated this to numbers earlier for Outlet_Size
mean_imputer = SimpleImputer(strategy='mean') #for Item_Weight

In [88]:
scaler = StandardScaler()
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [89]:
num_pipe = make_pipeline(mean_imputer,scaler)
num_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [80]:
cat_pipe = make_pipeline(ohe)
cat_pipe

Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [90]:
ord_pipe = make_pipeline(most_frequent_imputer,scaler) #made this extra pipeline due to updates from above and I wanted to have a mean average for Item Weight
ord_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('standardscaler', StandardScaler())])

In [91]:
num_tuple = (num_pipe, num_cols)
cat_tuple = (cat_pipe, cat_selector)
ord_tuple = (ord_pipe, ord_cols)

In [92]:
preprocessor = make_column_transformer(num_tuple,cat_tuple,ord_tuple)
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Item_Weight', 'Item_Visibility', 'Item_MRP',
                                  'Outlet_Establishment_Year']),
                                ('pipeline-2',
                                 Pipeline(steps=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f432a9df250>),
                                ('pipeline-3',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Item_Fat_Content', 'Outlet_Size'])])

In [93]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Item_Weight', 'Item_Visibility', 'Item_MRP',
                                  'Outlet_Establishment_Year']),
                                ('pipeline-2',
                                 Pipeline(steps=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f432a9df250>),
                                ('pipeline-3',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Item_Fat_Content', 'Outlet_Size'])])

In [94]:
#transform
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [95]:
# Check for missing values and that data is scaled and one-hot encoded
#This code was extracted from LP 
print(np.isnan(X_train_processed).sum().sum(), 'missing values in training data')
print(np.isnan(X_test_processed).sum().sum(), 'missing values in testing data')
print('\n')
print('All data in X_train_processed are', X_train_processed.dtype)
print('All data in X_test_processed are', X_test_processed.dtype)
print('\n')
print('shape of data of X_train is', X_train_processed.shape)
print('shape of data X_test is', X_test_processed.shape)
print('\n')
X_train_processed



0 missing values in training data
0 missing values in testing data


All data in X_train_processed are float64
All data in X_test_processed are float64


shape of data of X_train is (6392, 29)
shape of data X_test is (2131, 29)




array([[ 0.81724868, -0.71277507,  1.82810922, ...,  0.        ,
        -0.7403206 ,  0.28737371],
       [ 0.5563395 , -1.29105225,  0.60336888, ...,  0.        ,
         1.35076614,  0.28737371],
       [-0.13151196,  1.81331864,  0.24454056, ...,  0.        ,
         1.35076614,  0.28737371],
       ...,
       [ 1.11373638, -0.92052713,  1.52302674, ...,  0.        ,
        -0.7403206 ,  0.28737371],
       [ 1.76600931, -0.2277552 , -0.38377708, ...,  0.        ,
        -0.7403206 ,  0.28737371],
       [ 0.81724868, -0.95867683, -0.73836105, ...,  0.        ,
        -0.7403206 , -1.38404828]])